
### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2020 Semester 1

## Assignment 1: Naive Bayes Classifiers

###### Submission deadline: 7 pm, Monday 20 Apr 2020

**Student Name(s):**    `Alec Yu, Michael Jaworski`

**Student ID(s):**     `993433, 833751`


This iPython notebook is a template which you will use for your Assignment 1 submission.

Marking will be applied on the four functions that are defined in this notebook, and to your responses to the questions at the end of this notebook (Submitted in a separate PDF file).

**NOTE: YOU SHOULD ADD YOUR RESULTS, DIAGRAMS AND IMAGES FROM YOUR OBSERVATIONS IN THIS FILE TO YOUR REPORT (the PDF file).**

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find.

**Adding proper comments to your code is MANDATORY. **

In [79]:
import os
import pandas as pd
import numpy as np
# count attribute values
from collections import Counter
from math import sqrt, e, pi, log
from random import sample
import matplotlib.pyplot as plt

In [538]:
def get_filenames(location: str):
    
    # returns filenames from datasets folder location
    
    files = os.listdir(location)
    
    return [filename.replace(".data", "") for filename in files if ".data" in filename]
    

def read_data(filename: str):  
    
    # reads the filename.data data file and the filename.h header file and return a dataframe 
    
    df = pd.read_csv(f"datasets/{filename}.data", header=None)
    header = open(f"datasets/{filename}.h", "r").read().split(",")
    df.columns = header
    
    return df

def get_dtypes(filename: str):
    
    dtypes = {}
    
    rows = open(f"datasets/{filename}.dtypes.txt").read().split("\n")
    
    for row in rows:
        attribute = row.split(": ")[0]
        dtype = row.split(": ")[1]
        dtypes[attribute] = dtype
    
    return dtypes

def replace_all(df, d):
    
    # replace multiple strings (i.e. '?' and 'unknown' with np.nan)
    
    for k, v in d.items():
        df = df.replace(k, v)
    
    return df

In [25]:
# # find nan encoding i.e. '?' or 'unknown' 

# # get filenames
# filenames = get_filenames("datasets")

# attribute_values_list = []

# for filename in filenames:
    
#     df = read_data(filename)
    
#     for column in df.columns[:-1]:
#         attribute_values = df[column].tolist()
#         attribute_values_list.append(attribute_values)
        
# # sort attribute values by frequency

# attribute_values = [value for attribute_values in attribute_values_list for value in attribute_values]
# attribute_value_counter = Counter(attribute_values)
# attribute_value_counter = {k:v for k, v in sorted(attribute_value_counter.items(), key = lambda item: item[1], reverse = True)}

hence, 'unknown' and '?' represent missing values... replace missing values with np.nan.

In [548]:
# This function should prepare the data by reading it from a file and converting it into a useful format for training and testing

def preprocess(df, filename):
    
    # replace missing values with np.nan
    
    replacements = {"unknown": np.nan, "?": np.nan}  
    df = replace_all(df, replacements)
    
    # drop 'na' columns
    df = df.dropna(axis = 'columns')
    
    # rearrange class to [-1] position
    CLASS = df.pop("CLASS").astype("category")
    df["CLASS"] = CLASS
    
    # correct dtypes
    
    nominal_files = ["breast-cancer-wisconsin", "mushroom", "lymphography"]
    numeric_files = ["wdbc", "wine"]
    ordinal_files = ["car", "nursery", "somerville"]
    mixed_files = ["adult", "bank", "university"]
    
    if (filename in nominal_files):
        datatype = "nominal"
        for column in df.columns[:-1]:
            df[column] = df[column].astype("category")
            
    elif (filename in ordinal_files):
        datatype = "ordinal"
        for column in df.columns[:-1]:
            df[column] = df[column].astype("category")
        
    elif (filename in numeric_files):
        datatype = "numeric"
        for column in df.columns[:-1]:
            df[column] = pd.to_numeric(df[column])
        
    elif (filename in mixed_files):
        datatype = "mixed"
        dtypes = get_dtypes(filename)
        for column in df.columns[:-1]:
            try:
                dtype = dtypes[column]
                if dtype == "categoric":
                    df[column] = df[column].astype("category")
                if dtype == "numeric":
                    df[column] = pd.to_numeric(df[column])
            except:
                pass
            
    return df

In [27]:
filename = "university"
df = read_data(filename)
df.head()

,university-name,state,control,number-of-students,male:female-ratio,student:faculty-ratio,sat-veral,sat-math,expenses,percent-financial-aid,number-of-applicants,percent-admittance,percent-enrolled,CLASS,social,quality-of-life,academic-emphasis
0,adelphi,newyork,private,5-10,0.30,15.0,500,475,7-10,60,4-7,70,40,2,2,2,biology
1,arizona-state,arizona,state,20+,0.50,20.0,450,500,4-7,50,17+,80,60,3,4,5,fine-arts
2,boston-college,massachusetts,private:roman-catholic,5-10,0.40,20.0,500,550,10+,60,10-13,50,40,4,5,3,english
3,boston-university,massachusetts,private,10-15,0.45,12.0,550,575,10+,60,13-17,60,40,4,4,3,liberal-arts
4,brown,rhodeisland,private,5-,0.50,11.0,625,650,10+,40,10-13,20,50,5,4,5,arts:sciences


In [28]:
# k_means discretization function.
# optimise_k and get_wss function used to determine optimal k value.

def k_means(values, k, old_centroids = None):

    # 1. Select k points at random to act as seed centroids
    # 2. Assign each instance to the cluster with nearest
    # centroid
    # 3. Recompute centroids of the clusters using current
    # assignment. Centroid = centre or mean point of cluster
    # 4. Repeat step 2 until the assignment of instances to
    # clusters is stable
    
    if old_centroids == None:
        centroids = sample(list(set(values)), k)
    else:
        centroids = old_centroids    
        
    clusters = [None for val in range(len(values))]

    for i in range(len(values)):
        value = values[i]

        # initialize distance to 'infinity' and cluster to 'none'
        distance = np.inf
        cluster = None

        for j in range(len(centroids)):

            centroid = centroids[j]

            if abs(centroid - value) < distance:

                distance = abs(centroid - value)

                clusters[i] = j  

    new_centroids = [[] for i in range(k)]

    for i in range(len(clusters)):
        cluster = clusters[i]
        new_centroids[cluster].append(values[i])
    
    new_centroids = list(map(np.mean, new_centroids))

    if set(new_centroids) == set(centroids):
        return clusters, centroids
    else:
        clusters, centroids = k_means(values, k, new_centroids)
    
    return clusters, centroids

def get_wss(values, clusters, centroids):
    
    se = []

    for i in range(len(values)):
        se.append((centroids[clusters[i]] - values[i]) ** 2)

    wss = sum(se)

    return wss

def optimize_k(values, ks):
    
    wsss = {}
    
    for k in ks:
        clusters, centroids = k_means(values, k)
        wss = get_wss(values, clusters, centroids)
        wsss[k] = wss
        
    return wsss

testing optimize_k() function

In [29]:
# kss = [3, 5, 10, 50, 100]
# wsss = optimize_k(values, kss)
# plt.plot(list(wsss.keys()), list(wsss.values()))
# plt.title("elbow")

In [90]:
df

,buying,maint,doors,persons,lug_boot,safety,CLASS,predictions
0,vhigh,vhigh,2,2,small,low,unacc,unacc
1,vhigh,vhigh,2,2,small,med,unacc,unacc
2,vhigh,vhigh,2,2,small,high,unacc,unacc
3,vhigh,vhigh,2,2,med,low,unacc,unacc
4,vhigh,vhigh,2,2,med,med,unacc,unacc
...,...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good,good
1724,low,low,5more,more,med,high,vgood,vgood
1725,low,low,5more,more,big,low,unacc,unacc
1726,low,low,5more,more,big,med,good,good


In [583]:
# for naive bayes

def get_categoric_priors(df):
    
    # returns dataframe of categorical attributes' prior probabilities
    
    priors = []
    
    for column in df.columns[:-1]:
        
        # create "n" column to be able to aggregate attributes
        
        df2 = df.copy()
        df2["n"] = 1
        
        # get count of classes and count of attribute given class
        
        attributes = df2.groupby(["CLASS", column]).agg({"n": "count"}).reset_index()
        classes = attributes.groupby(["CLASS"])["n"].sum().reset_index().rename(columns = {"n": "total"})
        prior = pd.merge(attributes, classes, on = "CLASS", how = "left")
        
        # create "attribute" column to be able to concat prior dataframes
        
        prior["attribute"] = column
        prior = prior.rename(columns = {column: "attribute_value"})
        
        # get P(attribute|class)
        
        prior["p"] = prior.n / prior.total
        prior = prior[["CLASS", "attribute", "attribute_value", "n", "total", "p"]]
        
        priors.append(prior)
        
    priors = pd.concat(priors)
        
    return priors

# for gnb

def get_numeric_statistics(df):
    
    # returns dataframe of numerical attributes' mean and standard deviation
    
    numerical_statistics = df.agg(["mean", "std"]).transpose().reset_index().rename(columns = {"index": "attribute"})
    
    return numerical_statistics

def gaussian_pdf(x, mean, std):
    
    # gaussian function used to determine P(X = x|C) 
    
    return (1/(std * sqrt(2 * pi))) * e ** (-1/2 * ((x - mean)/std) ** 2)


# class priors

def get_class_priors(df):
    
    # returns the probability of each class occuring
    
    prior = df.groupby(["CLASS"]).size().reset_index().rename(columns = {0: "n"})
    prior["total"] = sum(prior["n"])
    
    # get P(class)
    
    prior["p"] = prior.n / prior.total
    
    return prior

In [640]:
# This function should calculat prior probabilities and likelihoods from the training data and using
# them to build a naive Bayes model

def train(df):

    class train_model:
        def __init__(self, class_priors, categoric_priors, numeric_statistics):
            
            self.class_priors = class_priors
            self.categoric_priors = categoric_priors
            self.numeric_statistics = numeric_statistics

        def gaussian_pdf(self, x, mean, std):
            # gaussian function used to determine P(X = x|C) 
            return (1/(std * sqrt(2 * pi))) * e ** (-1/2 * ((x - mean)/std) ** 2)

        def predict(self, df):
            
            # epsilon smoothing value
            epsilon = 1/(df.size + 1)
            
            predictions = []
            
            dtypes = df.dtypes.apply(lambda x: x.name).to_dict()
            
            CLASSES = model.class_priors.CLASS.tolist()
            
            class_ps = {}
            
            for row_number, row in df.iterrows():

                for CLASS in CLASSES:

                    attribute_ps = {}

                    for index in row.index[:-1]:

                        attribute = index 
                        attribute_value = row[index]

                        if dtypes[attribute] in categoric:

                            attribute_p = model.categoric_priors.loc[
                                (model.categoric_priors["CLASS"] == CLASS) & \
                                (model.categoric_priors["attribute"] == attribute) & \
                                (model.categoric_priors["attribute_value"] == attribute_value)
                            ].p.values[0]
                            
                            # simple epsilon smoothing
                            if (attribute_p == 0):
                                attribute_p = epsilon
                            
                            attribute_ps[attribute] = attribute_p

                        elif dtypes[attribute] in numeric:

                            mean = model.numeric_statistics.loc[
                                model.numeric_statistics.attribute == attribute
                            ]["mean"].values[0]
                            
                            std = model.numeric_statistics.loc[
                                model.numeric_statistics.attribute == attribute
                            ]["std"].values[0]

                            attribute_p = model.gaussian_pdf(attribute_value, mean, std)
                            
                            # simple epsilon smoothing
                            if (attribute_p == 0):
                                attribute_p = epsilon
                            
                            attribute_ps[attribute] = attribute_p
                            
                        else:
                            attribute_ps[attribute] = epsilon

                    class_p = model.class_priors.loc[
                        model.class_priors["CLASS"] == CLASS
                    ].p.values[0]
                    
                    class_p = log(class_p)
                    
                    # for value in attribute_ps.values():
                    #     class_p *= value
                        
                    # logarithmic smoothing
        
                    for value in attribute_ps.values():
                        class_p += log(value)

                    class_ps[CLASS] = class_p

                prediction = list({k: v for k, v in sorted(class_ps.items(), key=lambda item: item[1], reverse=True)}.keys())[0]

                predictions.append(prediction)
                                   
            return predictions
    
    ###
    
    numeric = ['int64', 'float64']
    categoric = ['category']
    
    class_priors = get_class_priors(df)

    if len(df.select_dtypes(numeric).columns) > 1:
        numeric_statistics = get_numeric_statistics(df.select_dtypes(numeric))
    else:
        numeric_statistics = None
        
    if len(df.select_dtypes(categoric).columns) > 1:
        categoric_priors = get_categoric_priors(df.select_dtypes(categoric))
    else:
        categoric_priors = None
    
    model = train_model(class_priors, categoric_priors, numeric_statistics)
        
    return model

In [585]:
# This function should predict classes for new items in a test dataset (for the purposes of this assignment, you
# can re-use the training data as a test set

# Multiplying probs, need to take logs of probabilities
# Smoothe training probabilities
# Change format of the probabilities, and ways of accessing them
def predict(df, model):
    
    predictions = model.predict(df)
    
    return predictions

In [586]:
df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,CLASS
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [513]:
from random import shuffle, seed

In [587]:
def get_folds(df, size): 
    seed(2020)
    seq = list(range(df.shape[0]))
    shuffle(seq)
    folds = [seq[i::size] for i in range(size)]
    return folds

In [643]:
def cross_validation(df, k):
    
    folds = get_folds(df, k)

    metrics_list = []
    
    for i in range(len(folds)):
        
        folds2 = folds.copy()

        test_index = folds2.pop(i)
        train_index = [index for fold in folds2 for index in fold]

        test_df = df.iloc[test_index]
        train_df = df.iloc[train_index]

        model = train(train_df)

        predictions = predict(test_df, model)
        labels = test_df['CLASS'].tolist()
        
        metrics = evaluate(labels, predictions)
        
        print(f"cross validation fold {i} accuracy: {metrics}")

        metrics_list.append(metrics)
    
    metrics = np.mean(metrics_list)
    
    return metrics

In [644]:
def evaluate(labels, predictions):
    
    correct = 0;
    incorrect = 0;
    
    for i in range(0, len(labels)):
        if(labels[i] == predictions[i]):
            correct += 1
        else:
            incorrect += 1;

    accuracy = correct/(correct + incorrect)
    
    return accuracy

In [649]:
filename = "wdbc"

print(filename)
    
# read in data
df = read_data(filename)

# preprocess data
df = preprocess(df, filename)

wdbc


In [650]:
df = discretise(df)

In [651]:
model = train(df)

In [654]:
predict(df.iloc[1:10], model)

['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M']

In [639]:
model.categoric_priors.loc[model.categoric_priors.p == 0]

,CLASS,attribute,attribute_value,n,total,p
4,B,feat1,21.883111,0,357,0.0
5,M,feat1,9.707765,0,212,0.0
4,B,feat3,139.917391,0,357,0.0
5,M,feat3,61.607403,0,212,0.0
4,B,feat4,1904.307692,0,357,0.0
4,B,feat8,0.145514,0,357,0.0
5,M,feat8,0.011526,0,212,0.0
4,B,feat11,2.710000,0,357,0.0
3,B,feat13,7.984364,0,357,0.0
4,B,feat13,20.315000,0,357,0.0


In [613]:
test = read_data("wdbc")

In [620]:
test.groupby(["CLASS"]).size()

CLASS
B    357
M    212
dtype: int64

## Questions 


If you are in a group of 1, you will respond to question (1), and **one** other of your choosing (two responses in total).

If you are in a group of 2, you will respond to question (1) and question (2), and **two** others of your choosing (four responses in total). 

A response to a question should take about 100–250 words, and make reference to the data wherever possible.

#### NOTE: you may develope codes or functions in respond to the question, but your formal answer should be added to a separate file.

### Q1
Try discretising the numeric attributes in these datasets and treating them as discrete variables in the na¨ıve Bayes classifier. You can use a discretisation method of your choice and group the numeric values into any number of levels (but around 3 to 5 levels would probably be a good starting point). Does discretizing the variables improve classification performance, compared to the Gaussian na¨ıve Bayes approach? Why or why not?

In [645]:
def discretise(df):
    
    # after preprocess function
    
    numeric = ['int64', 'float64']
    
    for column in df.select_dtypes(numeric).columns:
        bins, centroids = k_means(df[column].tolist(), 5)
        df[column] = [centroids[bin] for bin in bins]
        df[column] = df[column].astype("category")
        
    return df

In [646]:
nominal_files = ["breast-cancer-wisconsin", "mushroom", "lymphography"]
numeric_files = ["wdbc", "wine"]
ordinal_files = ["car", "nursery", "somerville"]
mixed_files = ["adult", "bank", "university"]

metrics_df_list = []

for filename in numeric_files:
    
    print(filename)
    
    # read in data
    df = read_data(filename)

    # preprocess data
    df = preprocess(df, filename)

    # discretise df
    df2 = df.copy()
    df2 = discretise(df2)

    metrics = cross_validation(df, 5)
    metrics2 = cross_validation(df2, 5)

    metrics_df = pd.DataFrame(data = {"filename": [filename], "accuracy": [metrics], "accuracy_discretised": [metrics2]})
    
    metrics_df_list.append(metrics_df)
    
metrics_df = pd.concat(metrics_df_list)

wdbc
cross validation fold 0 accuracy: 0.7105263157894737
cross validation fold 1 accuracy: 0.5964912280701754
cross validation fold 2 accuracy: 0.631578947368421
cross validation fold 3 accuracy: 0.6228070175438597
cross validation fold 4 accuracy: 0.5752212389380531
cross validation fold 0 accuracy: 0.9649122807017544
cross validation fold 1 accuracy: 0.9385964912280702
cross validation fold 2 accuracy: 0.9298245614035088
cross validation fold 3 accuracy: 0.9035087719298246
cross validation fold 4 accuracy: 0.9380530973451328
wine
cross validation fold 0 accuracy: 0.3888888888888889
cross validation fold 1 accuracy: 0.3888888888888889
cross validation fold 2 accuracy: 0.4722222222222222
cross validation fold 3 accuracy: 0.42857142857142855
cross validation fold 4 accuracy: 0.3142857142857143
cross validation fold 0 accuracy: 1.0
cross validation fold 1 accuracy: 0.9722222222222222
cross validation fold 2 accuracy: 0.9722222222222222
cross validation fold 3 accuracy: 1.0
cross validat

In general, for the numeric datasets. It seems the naiva bayes performs better than the Gaussian Naive Bayes. This is likely because the numeric attributes violate the assumption that the data is normally distributed.

### Q2
Implement a baseline model (e.g., random or 0R) and compare the performance of the na¨ıve Bayes classifier to this baseline on multiple datasets. Discuss why the baseline performance varies across datasets, and to what extent the na¨ıve Bayes classifier improves on the baseline performance.

### Q3
Since it’s difficult to model the probabilities of ordinal data, ordinal attributes are often treated as either nominal variables or numeric variables. Compare these strategies on the ordinal datasets provided. Deterimine which approach gives higher classification accuracy and discuss why.

### Q4
Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out or cross–validation evaluation strategy (you should implement this yourself and do not simply call existing implementations from `scikit-learn`). How does your estimate of effectiveness change, compared to testing on the training data? Explain why. (The result might surprise you!)

### Q5
Implement one of the advanced smoothing regimes (add-k, Good-Turing). Does changing the smoothing regime (or indeed, not smoothing at all) affect the effectiveness of the na¨ıve Bayes classifier? Explain why, or why not.

### Q6
The Gaussian na¨ıve Bayes classifier assumes that numeric attributes come from a Gaussian distribution. Is this assumption always true for the numeric attributes in these datasets? Identify some cases where the Gaussian assumption is violated and describe any evidence (or lack thereof) that this has some effect on the NB classifier’s predictions.

On the numeric datasets, performance is much better with Naive Bayes (categorical) compared to GNB.

In [664]:
metrics_df

,filename,accuracy,accuracy_discretised
0,wdbc,0.627325,0.934979
0,wine,0.398571,0.988889


can also plot the variables...